In [1]:
import json
from portfolio import Portfolio
from optimizer import PortfolioOptimizer
from simulations import MonteCarloSimulation
from utils import (
    get_simulation_insights,
    display_optimal_weights
)

#Snowpark lib
from snowflake.snowpark import Session
import pandas as pd
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

config = {
    "TICKERS" : [], "START_DATE" : '2020-01-01', "END_DATE" : '2023-01-01', "INITIAL_INVESTMENT" : 100000,
    "NUM_SIMULATIONS" : 10000, "TIME_HORIZON" : 252, "RISK_FREE_RATE" : 0.02, "WEIGHTS" : None, "OPTIMIZE" : True, "BALANCED" : False
}

In [2]:
table_name = 'STOCKS_DATA'
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [3]:
tickers = ["MTFS", "ALPA", "OOGGL", "TCA", "ABC"]
config["TICKERS"] = tickers

In [4]:
def load_data(df, tickers: list):
    stock_data = {}
    min_len = []
    for ticker in tickers:
        data = df[df["STOCK"] == ticker]
        stock_data[ticker] = list(data["ADJ_CLOSE"])
        min_len.append(len(data))
    last_len = min(min_len)
    for ticker in tickers:
        stock_data[ticker] = stock_data[ticker][:last_len]
    
    stock_data = pd.DataFrame(stock_data)
    return stock_data

# load_data(df, config["TICKERS"]).head()

In [6]:
def main(config=config):
    
    # Extract configuration parameters
    tickers = config.get('TICKERS', ["MTFS", "ALPA", "OOGGL"])
    start_date = config.get('START_DATE')
    end_date = config.get('END_DATE')
    initial_investment = config.get('INITIAL_INVESTMENT', 1000)
    num_simulations = config.get('NUM_SIMULATIONS', 10000)
    time_horizon = config.get('TIME_HORIZON', 252)
    risk_free_rate = config.get('RISK_FREE_RATE', 0.0)
    custom_weights = config.get('WEIGHTS')
    optimization_config = config.get('optimization', {})
    optimize = optimization_config.get('OPTIMIZE', True)
    balanced = optimization_config.get('BALANCED', False)
    # Load data
    stock_data = load_data(df, tickers)
    
    # Create portfolio
    portfolio = Portfolio(stock_data)
    portfolio.calculate_returns()
    
    # Annualize returns and covariance
    expected_returns = portfolio.returns.mean() * 252
    covariance_matrix = portfolio.returns.cov() * 252

    # return covariance_matrix, expected_returns
    
    # Determine weights
    if optimize:
        optimizer = PortfolioOptimizer(
            expected_returns,
            covariance_matrix,
            risk_free_rate=risk_free_rate
        )
        if balanced:
            optimal_weights = optimizer.minimize_volatility(target_return=expected_returns.mean())
            print("\nOptimal Balanced Portfolio Weights:")
        else:
            optimal_weights = optimizer.maximize_sharpe_ratio()
            print("\nOptimal Portfolio Weights to Maximize Sharpe Ratio:")
        display_optimal_weights(stock_data.columns, optimal_weights)
        weights = optimal_weights
    elif custom_weights:
        weights = custom_weights
        print("\nUsing Custom Weights:")
        display_optimal_weights(stock_data.columns, weights)
    else:
        num_assets = len(expected_returns)
        weights = [1.0 / num_assets] * num_assets
        print("\nUsing Equal Weights:")
        display_optimal_weights(stock_data.columns, weights)
    
    # Perform Monte Carlo Simulation
    simulation = MonteCarloSimulation(portfolio.returns, initial_investment, weights)
    all_cumulative_returns, final_portfolio_values = simulation.run_simulation(
        num_simulations, time_horizon
    )
    
    # Analyze Results
    final_insights = get_simulation_insights(final_portfolio_values, initial_investment)
    print("**************************")
    for k, v in final_insights.items():
        print(k,":", v)
    # print(final_insights)
    return all_cumulative_returns, final_portfolio_values
    
    # Plot results
    # plot_simulation_results(all_cumulative_returns, final_portfolio_values)



a, b = main()


Optimal Portfolio Weights to Maximize Sharpe Ratio:
  Ticker  Weight
0   MTFS  0.3105
1   ALPA  0.5264
2  OOGGL  0.0000
3    TCA  0.1631
4    ABC  0.0000
**************************
Initial Investment : $100,000.00
Expected Final Portfolio Value : $128,642.51
Median Final Portfolio Value : $125,024.50
Standard Deviation of Final Portfolio Value : $29,996.20
Value at Risk (VaR 95%) : $13,636.10
Conditional Value at Risk (CVaR 95%) : $21,356.19
Probability of Loss : 16.40%
Sharpe Ratio : 0.9549


In [9]:
list(a[0])

[102173.65659044632,
 97595.58426611996,
 102315.65685849763,
 99763.27464470608,
 99432.22975044047,
 99800.99814499165,
 99692.6137475191,
 99360.98678397872,
 98215.73335859978,
 100622.11876283758,
 99528.0863666888,
 102137.8325483164,
 103239.68061301789,
 100839.57324659276,
 101230.76025284572,
 101505.02840237797,
 99719.49965983831,
 98529.33533877887,
 100542.66289175357,
 96947.93767025539,
 98789.54830046606,
 101902.7409361311,
 101005.89647562524,
 99837.07945208768,
 102403.72190200884,
 98586.61777146808,
 100810.05474989372,
 100904.2238463748,
 100240.54137357898,
 98700.4197250561,
 103242.99327527772,
 101378.31637207247,
 100551.59894216877,
 99531.25135023092,
 100388.55833825057,
 98770.53341259902,
 100220.72052292581,
 99028.14975888567,
 100205.31226903117,
 101959.69351644447,
 100089.44514275671,
 101111.88840039505,
 99521.33271963676,
 99539.4773055715,
 101161.66802170934,
 99259.6484899727,
 97051.17094331297,
 99463.73435162648,
 99635.79901665357,
 10